In [2]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD
print(os.listdir("/kaggle/input/flowers-recognition/flowers"))

['dandelion', 'daisy', 'sunflower', 'tulip', 'rose']


In [3]:
# define parameters
CLASS_NUM = 5
BATCH_SIZE = 16
EPOCH_STEPS = int(4323/BATCH_SIZE)
IMAGE_SHAPE = (224, 224, 3)
IMAGE_TRAIN = '/kaggle/input/flowers-recognition/flowers'
myGoogleNet = 'googlenet_flowers.h5'

In [4]:
# prepare data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    #rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [5]:
generator_main = train_datagen.flow_from_directory(
    IMAGE_TRAIN,
    target_size=(IMAGE_SHAPE[0], IMAGE_SHAPE[1]),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 4317 images belonging to 5 classes.


In [6]:
# generator_main = tf.keras.utils.image_dataset_from_directory(
#   IMAGE_TRAIN,
#   validation_split=0.2,
#   subset="validation",
#   seed=123,
#   image_size=(IMAGE_SHAPE[0], IMAGE_SHAPE[1]),
#   batch_size=BATCH_SIZE)

In [7]:
def my_generator(generator):
    while True: # keras requires all generators to be infinite
        data = next(generator)
        x = data[0]
        y = data[1], data[1], data[1]
        yield x, y

train_generator = my_generator(generator_main)

In [8]:
# create model
def inception(x, filters):
    # 1x1
    path1 = Conv2D(filters=filters[0], kernel_size=(1,1), strides=1, padding='same', activation='relu')(x)

    # 1x1->3x3
    path2 = Conv2D(filters=filters[1][0], kernel_size=(1,1), strides=1, padding='same', activation='relu')(x)
    path2 = Conv2D(filters=filters[1][1], kernel_size=(3,3), strides=1, padding='same', activation='relu')(path2)
    
    # 1x1->5x5
    path3 = Conv2D(filters=filters[2][0], kernel_size=(1,1), strides=1, padding='same', activation='relu')(x)
    path3 = Conv2D(filters=filters[2][1], kernel_size=(5,5), strides=1, padding='same', activation='relu')(path3)

    # 3x3->1x1
    path4 = MaxPooling2D(pool_size=(3,3), strides=1, padding='same')(x)
    path4 = Conv2D(filters=filters[3], kernel_size=(1,1), strides=1, padding='same', activation='relu')(path4)

    return Concatenate(axis=-1)([path1,path2,path3,path4])


In [9]:
def auxiliary(x, name=None):
    layer = AveragePooling2D(pool_size=(5,5), strides=3, padding='valid')(x)
    layer = Conv2D(filters=128, kernel_size=(1,1), strides=1, padding='same', activation='relu')(layer)
    layer = Flatten()(layer)
    layer = Dense(units=256, activation='relu')(layer)
    layer = Dropout(0.4)(layer)
    layer = Dense(units=CLASS_NUM, activation='softmax', name=name)(layer)
    return layer

In [10]:
def googlenet():
    layer_in = Input(shape=IMAGE_SHAPE)
    
    # stage-1
    layer = Conv2D(filters=64, kernel_size=(7,7), strides=2, padding='same', activation='relu')(layer_in)
    layer = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(layer)
    layer = BatchNormalization()(layer)

    # stage-2
    layer = Conv2D(filters=64, kernel_size=(1,1), strides=1, padding='same', activation='relu')(layer)
    layer = Conv2D(filters=192, kernel_size=(3,3), strides=1, padding='same', activation='relu')(layer)
    layer = BatchNormalization()(layer)
    layer = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(layer)

    # stage-3
    layer = inception(layer, [ 64,  (96,128), (16,32), 32]) #3a
    layer = inception(layer, [128, (128,192), (32,96), 64]) #3b
    layer = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(layer)
    
    # stage-4
    layer = inception(layer, [192,  (96,208),  (16,48),  64]) #4a
    aux1  = auxiliary(layer, name='aux1')
    layer = inception(layer, [160, (112,224),  (24,64),  64]) #4b
    layer = inception(layer, [128, (128,256),  (24,64),  64]) #4c
    layer = inception(layer, [112, (144,288),  (32,64),  64]) #4d
    aux2  = auxiliary(layer, name='aux2')
    layer = inception(layer, [256, (160,320), (32,128), 128]) #4e
    layer = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(layer)
    
    # stage-5
    layer = inception(layer, [256, (160,320), (32,128), 128]) #5a
    layer = inception(layer, [384, (192,384), (48,128), 128]) #5b
    layer = AveragePooling2D(pool_size=(7,7), strides=1, padding='valid')(layer)
    
    # stage-6
    layer = Flatten()(layer)
    layer = Dropout(0.4)(layer)
    layer = Dense(units=256, activation='linear')(layer)
    main = Dense(units=CLASS_NUM, activation='softmax', name='main')(layer)
    
    model = Model(inputs=layer_in, outputs=[main, aux1, aux2])
    
    return model

#temporary edit

In [12]:
model = tf.keras.models.load_model('/kaggle/input/googlenetpretrainedmodel/googleNetFlower/googlenet_flower.h5')

2022-12-04 11:46:19.000392: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [13]:
# train model
# model = googlenet()
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_118 (Conv2D)             (None, 112, 112, 64) 9472        input_3[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_26 (MaxPooling2D) (None, 56, 56, 64)   0           conv2d_118[0][0]                 
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 56, 56, 64)   256         max_pooling2d_26[0][0]           
____________________________________________________________________________________________

In [ ]:
model.save(myGoogleNet)

In [ ]:
# model.load_weights(MODEL_NAME)

In [ ]:
# optimizer = Adam(learning_rate=0.000001, name="Adam")
epochs = [50, 50, 50, 50]
history_all = {}

In [ ]:
model.compile(loss='categorical_crossentropy', 
                  loss_weights={'main': 1.0, 'aux1': 0.3, 'aux2': 0.3},
                  optimizer = Adam(learning_rate=0.000001, name="Adam"), metrics=['accuracy'])
# model.fit_generator(
#             train_generator,
#             steps_per_epoch=EPOCH_STEPS,
#             epochs=5,
#             #callbacks=[checkpoint]
#             shuffle=False
#             )
model.fit(x=train_generator, y=None, batch_size=10, epochs=10, verbose='auto', 
          callbacks=None, validation_split=None, validation_data=None, 
          shuffle=False, class_weight=None, sample_weight=None, 
          initial_epoch=0, steps_per_epoch=EPOCH_STEPS)


In [ ]:
# for i in range(len(optimizer)):
#     print('Usnig optimizer: ' + optimizer[i] + ', Epoch: ' + str(epochs[i]))
    
#     model.compile(loss='categorical_crossentropy', 
#                   loss_weights={'main': 1.0, 'aux1': 0.3, 'aux2': 0.3},
#                   optimizer=optimizer[i], metrics=['accuracy'])
    
#     train_history = model.fit_generator(
#             train_generator,
#             steps_per_epoch=EPOCH_STEPS,
#             epochs=epochs[i],
#             #callbacks=[checkpoint]
#             shuffle=True
#             )
    
#     # save history    
#     if len(history_all) == 0:
#         history_all = {key: [] for key in train_history.history}
    
#     for key in history_all:
#         history_all[key].extend(train_history.history[key])


In [ ]:
model.save(myGoogleNet)